<big><b>Step 1:</b> Load the SQL extension</big>

In [1]:
%load_ext sql

<big><b>Step 2</b>: Connect sqlite to coursedata database</big>

In [2]:
%%sql

sqlite:///CourseData.db

'Connected: @CourseData.db'

<big><b>Step 3</b>: Determine columns within the dataset so that we can import this information to the appropriate tables </big>

In [3]:
%%sql

PRAGMA table_info(DATASET3);

 * sqlite:///CourseData.db
Done.


cid,name,type,notnull,dflt_value,pk
0,term,TEXT,0,None,0
1,crn,TEXT,0,None,0
2,catalog_id,TEXT,0,None,0
3,section,TEXT,0,None,0
4,credits,TEXT,0,None,0
5,title,TEXT,0,None,0
6,meetings,TEXT,0,None,0
7,timecodes,TEXT,0,None,0
8,primary_instructor,TEXT,0,None,0
9,cap,TEXT,0,None,0


<big><b>Step 4:</b></big>

<big> A) Create INSTRUCTOR table </big>

In [4]:
%%sql 

DROP TABLE IF EXISTS INSTRUCTOR;
CREATE TABLE INSTRUCTOR(
    IID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> B) Create LOCATION table </big>

In [5]:
%%sql

DROP TABLE IF EXISTS LOCATION;
CREATE TABLE LOCATION(
    LID INTEGER PRIMARY KEY,
    Location TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> C) Create PROGRAM table</big>

In [6]:
%%sql

DROP TABLE IF EXISTS PROGRAM;
CREATE TABLE PROGRAM(
    PID INTEGER PRIMARY KEY,
    ProgramCode TEXT NOT NULL,
    ProgramName TEXT NOT NULL
);
    

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> D) Create CATALOG_COURSE table </big>

In [7]:
%%sql 

DROP TABLE IF EXISTS CATALOG_COURSE;
CREATE TABLE CATALOG_COURSE(
    CAID INTEGER NOT NULL PRIMARY KEY,
    CatalogID TEXT NOT NULL, 
    ProgramName TEXT NOT NULL,
    Credits VARCHAR NOT NULL,
    Title TEXT NOT NULL,
    PreReqs TEXT,
    Attributes TEXT,
    Description TEXT,
    FOREIGN KEY (ProgramName) REFERENCES PROGRAM(ProgramName)
); 


 * sqlite:///CourseData.db
Done.
Done.


[]

<big> E) Create OFFERINGS_COURSE table </big>

In [8]:
%%sql 

DROP TABLE IF EXISTS OFFERINGS_COURSE;
CREATE TABLE OFFERINGS_COURSE(
    COID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    Meetings TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    CRN TEXT NOT NULL,
    Term TEXT NOT NULL,
    Section TEXT NOT NULL,
    Cap INTEGER NOT NULL,
    Act INTEGER NOT NULL,
    Rem INTEGER NOT NULL,
    FOREIGN KEY (Name) REFERENCES INSTRUCTOR (Name),
    FOREIGN KEY (CatalogID) REFERENCES CATALOG_COURSE (CatalogID)
);
    

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> F) Create MEETINGS_COURSE table </big>

In [9]:
%%sql

DROP TABLE IF EXISTS MEETINGS_COURSE;
CREATE TABLE MEETINGS_COURSE(
    CMID INTEGER PRIMARY KEY,
    CRN TEXT NOT NULL,
    Location TEXT NOT NULL,
    Day TEXT NOT NULL,
    Start TEXT NOT NULL,
    End TEXT NOT NULL,
    FOREIGN KEY (Location) REFERENCES LOCATION(Location),
    FOREIGN KEY (CRN) REFERENCES OFFERINGS_COURSE(CRN)
);

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> <b> Step 5: </b> Populating the database with data

Here we populated the data with that we have imported into the 3 dataset tables

-Dataset 1 <br>
-Dataset 2 <br>
-Dataset 3 <br>
   </big>

<big> A) Populate PROGRAM table </big>

In [10]:
%%sql

DELETE FROM PROGRAM;
INSERT INTO PROGRAM (ProgramCode, ProgramName)
    SELECT DISTINCT program_code, program_name
    FROM DATASET1;

DELETE FROM PROGRAM
WHERE ProgramName = 'program_name';


 * sqlite:///CourseData.db
0 rows affected.
84 rows affected.
1 rows affected.


[]

<big> B) Populate INSTRUCTOR table </big>

In [19]:
%%sql

DELETE FROM INSTRUCTOR;
INSERT INTO INSTRUCTOR (Name)
    SELECT DISTINCT primary_instructor
    FROM DATASET3;
    
DELETE FROM INSTRUCTOR
WHERE Name = 'primary_instructor';

 * sqlite:///CourseData.db
(sqlite3.OperationalError) no such table: DATASET3
[SQL: INSERT INTO INSTRUCTOR (Name)
    SELECT DISTINCT primary_instructor
    FROM DATASET3;]
(Background on this error at: http://sqlalche.me/e/e3q8)


<big> C) Populate LOCATION table </big>

In [12]:
%%sql

DELETE FROM LOCATION;
INSERT INTO LOCATION (Location)
    SELECT DISTINCT location
    FROM DATASET2;
    
DELETE FROM LOCATION
WHERE Location = 'location';

 * sqlite:///CourseData.db
0 rows affected.
208 rows affected.
1 rows affected.


[]

<big> D) Populate CATALOG_COURSE table, link program name to PROGRAM table to ensure FK integrity </big>

In [13]:
%%sql

DELETE FROM CATALOG_COURSE;
INSERT INTO CATALOG_COURSE (CatalogID, ProgramName, Credits, Title, PreReqs, Attributes, Description)
    SELECT DISTINCT catalog_id, c.program_name, credits, course_title, prereqs, attributes, description
    FROM DATASET1 c
    LEFT JOIN PROGRAM d ON c.program_name = d.ProgramName;
    
DELETE FROM CATALOG_COURSE
WHERE CatalogID = 'catalog_id';

 * sqlite:///CourseData.db
0 rows affected.
2221 rows affected.
1 rows affected.


[]

<big> D) Populate OFFERINGS_COURSE table, link instructor name to INSTRUCTOR and catalog_id to CATALOG_COURSE table to ensure FK integrity </big>

In [14]:
%%sql

DELETE FROM OFFERINGS_COURSE;
INSERT INTO OFFERINGS_COURSE(Meetings, CRN, Term, CatalogID, Name, Section, Cap, Act, Rem)
    SELECT DISTINCT c.meetings, crn, term, c.catalog_id, primary_instructor, section, cap, act, rem
    FROM DATASET3 c
        INNER JOIN INSTRUCTOR i ON c.primary_instructor=i.Name
        INNER JOIN CATALOG_COURSE cc ON c.catalog_id = cc.CatalogID
 ;
DELETE FROM OFFERINGS_COURSE
WHERE CatalogID = 'catalog_id';


 * sqlite:///CourseData.db
0 rows affected.
15243 rows affected.
0 rows affected.


[]

<big> E) Populate MEETINGS_COURSE table, link location to LOCATION and CRN to OFFERINGS_COURSE table to ensure FK integrity </big>

In [15]:
%%sql

DELETE FROM MEETINGS_COURSE;
INSERT INTO MEETINGS_COURSE (CRN, Location, Day, Start, End)
    SELECT DISTINCT c.crn, c.location, day, start, end
    FROM DATASET2 c
    LEFT JOIN LOCATION l ON c.location=l.Location
    LEFT JOIN OFFERINGS_COURSE oc ON c.crn = oc.CRN;
    
DELETE FROM MEETINGS_COURSE
WHERE CRN = 'crn';


 * sqlite:///CourseData.db
0 rows affected.
284848 rows affected.
1 rows affected.


[]

In [18]:
%%sql

DROP TABLE IF EXISTS DATASET1;
DROP TABLE IF EXISTS DATASET2;
DROP TABLE IF EXISTS DATASET3;

vacuum;

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.


[]

In [6]:
%%sql


UPDATE OFFERINGS_COURSE
SET Name = (
    SELECT DISTINCT Name
    FROM INSTRUCTOR);


 * sqlite:///CourseData.db
15243 rows affected.


[]

In [7]:
%%sql

SELECT Name
FROM OFFERINGS_COURSE
LIMIT 10;

 * sqlite:///CourseData.db
Done.


Name
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
Michael P. Coyne
